Run the cell below to import the required packages:

In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import Pipeline

from nltk.corpus import stopwords
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#nltk.download('vader_lexicon')

Recall our example yesterday. We first used the Tfidf Vectorizer to preprocess the data and turn the words into a matrix where uncommon words were granted more weight:

In [2]:
articles = ['Football baseball basketball',
            'baseball giants cubs redsox',
            'football broncos cowboys',
            'baseball redsox tigers',
            'pop stars hendrix prince',
            'hendrix prince jagger rock',
            'joplin pearl jam tupac rock',
          ]

vectorizer = TfidfVectorizer(lowercase=True, 
                     token_pattern="\\b[a-zA-Z][a-zA-Z]+\\b", 
                     stop_words=stopwords.words('english'),
                     min_df=1)

X = vectorizer.fit_transform(articles).toarray()

pd.DataFrame(X,
             columns=vectorizer.get_feature_names())

,baseball,basketball,broncos,cowboys,cubs,football,giants,hendrix,jagger,jam,joplin,pearl,pop,prince,redsox,rock,stars,tigers,tupac
0,0.479185,0.675356,0.000000,0.000000,0.000000,0.560603,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.397106,0.000000,0.000000,0.000000,0.559675,0.000000,0.559675,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.464579,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.609819,0.609819,0.000000,0.506202,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.479185,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.560603,0.000000,0.000000,0.675356,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.451635,0.000000,0.000000,0.000000,0.000000,0.544082,0.451635,0.000000,0.000000,0.544082,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.473977,0.570997,0.000000,0.000000,0.000000,0.000000,0.473977,0.000000,0.473977,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.461804,0.461804,0.461804,0.000000,0.000000,0.000000,0.383337,0.000000,0.000000,0.461804


Then, we used an SVD to view the most important words making up each component:

In [3]:
svd = TruncatedSVD(2)
X_svd = svd.fit_transform(X)
pd.DataFrame(svd.components_.round(5),
             index = ["component_1","component_2"],
             columns = vectorizer.get_feature_names())

,baseball,basketball,broncos,cowboys,cubs,football,giants,hendrix,jagger,jam,joplin,pearl,pop,prince,redsox,rock,stars,tigers,tupac
component_1,0.59434,0.26389,0.10775,0.10775,0.25565,0.30849,0.25565,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.47627,0.00000,0.00000,0.31811,0.00000
component_2,-0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.51977,0.33357,0.10539,0.10539,0.10539,0.29259,0.51977,-0.00000,0.36438,0.29259,-0.00000,0.10539


Then we also wanted to scale our data using Normalizer. This ensures that each vector has a "norm" of 1. Vectors with a norm of 1 are easy to work with for calculating similarity. We see that each document is a linear combination of the components:

In [4]:
dtm_svd = Normalizer(copy=False).fit_transform(X_svd)

pd.DataFrame(dtm_svd.round(5),
             index=articles, 
             columns=["component_1","component_2" ])

,component_1,component_2
Football baseball basketball,1.0,-0.0
baseball giants cubs redsox,1.0,0.0
football broncos cowboys,1.0,0.0
baseball redsox tigers,1.0,-0.0
pop stars hendrix prince,0.0,1.0
hendrix prince jagger rock,0.0,1.0
joplin pearl jam tupac rock,0.0,1.0


### Pipelines

We can create a pipeline that performs all of the above processes. Let's make the pipe:

In [5]:
pipe = [('tfidf', TfidfVectorizer(stop_words='english', 
                        token_pattern="\\b[a-zA-Z][a-zA-Z]+\\b", 
                        min_df=2)),
       ('lsa', TruncatedSVD(2)),
       ('normalizer', Normalizer())]
pipeline = Pipeline(pipe)

Now we can put our article data through the pipe to generate the exact table that we made above. The code is a lot cleaner this way:

In [6]:
dtm_svd = pipeline.fit_transform(articles)
pd.DataFrame(dtm_svd.round(5),
             index=articles, 
             columns=["component_1","component_2" ])

,component_1,component_2
Football baseball basketball,1.0,0.0
baseball giants cubs redsox,1.0,-0.0
football broncos cowboys,1.0,0.0
baseball redsox tigers,1.0,-0.0
pop stars hendrix prince,-0.0,1.0
hendrix prince jagger rock,-0.0,1.0
joplin pearl jam tupac rock,-0.0,1.0


### 1.Topic Modeling

When we chose two components to use in our SVD, we were essentially choosing to use two topics. We can view the five most important words associated with each topic below:

In [7]:
n_topics = 2
n_words = 5
feature_names = vectorizer.get_feature_names()

for topic_num in range(n_topics):
    topic_mat = svd.components_[topic_num]

    print('Topic {}:'.format(topic_num + 1).center(80))

    topic_values = sorted(zip(topic_mat, feature_names), 
                          reverse=True)[:n_words]
    print(' '.join([y for x,y in topic_values]))
    print('-'*80)

                                    Topic 1:                                    
baseball redsox tigers football basketball
--------------------------------------------------------------------------------
                                    Topic 2:                                    
prince hendrix rock jagger stars
--------------------------------------------------------------------------------


### 2.Predictive Modeling

Here we will train a model on our outcome (Sports 1 or Music 0) and then use the model to predict on new sentences what we are talking about. First, let's add a logistic regression classifier to the end of our pipe:

In [9]:
model = LogisticRegression(solver="lbfgs")

pipeline = Pipeline(pipe + [('est', model)])

Now, we can use our previous articles as our training data and predict a topic with a test sentence:

In [11]:
pipeline.fit(articles, [1,1,1,1,0,0,0])

new_sentence = 'babe ruth played baseball'
y_pred_1 = pipeline.predict_proba([new_sentence])[:, 1]
print(y_pred_1)

[0.76068417]


Clearly, it makes sense that this new article was labeled as sports instead of music. What about a new sentence about music?

In [12]:
new_sentence = 'rock and roll music'
y_pred_1 = pipeline.predict_proba([new_sentence])[:, 1]
print(y_pred_1)

[0.31908412]


This would get labeled as music.

### 3.Article Suggestions

We can use a similar function to the one we used in our recommendation system unit to find articles most similar to a given article. Let's find the sentence most similar to Sentence 0:

In [16]:
def get_article_recommendations(articles, compare_article, df, num_recom):
    recs = []
    for article in range(len(df)):
        if article != compare_article:
            recs.append((np.dot(df[compare_article],df[article]), article))
            recs.sort(reverse = True)
    for i in range(num_recom):
        print(f"Article #{recs[i][1]}: {articles[recs[i][1]]}. Similarity score: {recs[i][0]}.")

article_num = 0
get_article_recommendations(articles, article_num,dtm_svd,5)

Article #3: baseball redsox tigers. Similarity score: 1.0.
Article #2: football broncos cowboys. Similarity score: 1.0.
Article #1: baseball giants cubs redsox. Similarity score: 1.0.
Article #5: hendrix prince jagger rock. Similarity score: 3.1899559897031195e-15.
Article #4: pop stars hendrix prince. Similarity score: 3.1428748374072194e-15.


It makes sense that Sentences 3, 2, and 1 are most similar to Sentence 0, as they are all sports-related.

### 4.Sentiment Analysis

Natural language processing can handle sentiment analysis. Polarity is near +1 for highly positive sentiment and near -1 for highly negative sentiment. 

Here's a highly positive sentence:

In [17]:
sid = SentimentIntensityAnalyzer()
print(sid.polarity_scores("Oh my god I love football, it's so awesome."))

{'neg': 0.0, 'neu': 0.302, 'pos': 0.698, 'compound': 0.9107}


Here's a highly negative sentence:

In [18]:
print(sid.polarity_scores("I hate swimming it makes me so tired."))

{'neg': 0.598, 'neu': 0.402, 'pos': 0.0, 'compound': -0.8147}
